In [1]:
from autogluon.tabular import TabularPredictor
import pandas as pd
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import *
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold


d:\Applications\anaconda3\envs\ag_new\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
feature_df = pd.read_csv('gaze_features_z_score.csv')
feature_df['binary_label'] = feature_df['Label'].apply(lambda x: 0 if x <= 4 else 1)
feature_df['ternary_label'] = feature_df['Label'].apply(lambda x: 0 if x <= 3 else 1 if x <= 6 else 2)
feature_df_binary = feature_df.drop(columns=['Label','Level','Participant','ternary_label'])
# feature_df_binary_distill = feature_df_binary[['binary_label','Fixation Count',
# #  'Fixation Dispersion Mean',
#  'Fixation Duration Min',
#  'Pupil Diameter Mean',
#  'Saccade Count',
#  'Saccade Duration Min']]

feature_df_ternary = feature_df.drop(columns=['Label','Level','Participant','binary_label'])
# feature_df_ternary_distill = feature_df_ternary[['ternary_label','Fixation Count',
# #  'Fixation Dispersion Mean',
#  'Fixation Duration Min',
#  'Pupil Diameter Mean',
#  'Saccade Count',
#  'Saccade Duration Min']]



In [56]:
# Adding synthetic cognitive load labels to the feature dataframe for demonstration purposes
np.random.seed(42)  # For reproducibility


# Splitting the dataset into training and testing sets
train_data_binary, test_data_binary = train_test_split(feature_df_binary, test_size=0.1, random_state=42)
train_data_ternary, test_data_ternary = train_test_split(feature_df_ternary, test_size=0.1, random_state=42)

predictor_binary_default = TabularPredictor(label='binary_label', eval_metric='accuracy', verbosity=2,path = 'AutogluonModels/ag-concentrate-binary').fit(train_data_binary) #CCS3project\AutogluonModels\ag-20241004_080342
predictor_ternary_default = TabularPredictor(label='ternary_label', eval_metric='accuracy', verbosity=2,path = 'AutogluonModels/ag-concentrate-ternary').fit(train_data_ternary) #CCS3project\AutogluonModels\ag-20241004_080400
# predictor = TabularPredictor.load("AutogluonModels/ag-20241004_000434")
# normalized: bi CCS3project\AutogluonModels\ag-20241004_103135 ter  CCS3project\AutogluonModels\ag-20241004_103156
# normalized wo z score: bi CCS3project\AutogluonModels\ag-20241004_103835 ter CCS3project\AutogluonModels\ag-20241004_103854
# only z score: bi CCS3project\AutogluonModels\ag-20241004_104224 ter  CCS3project\AutogluonModels\ag-20241004_104245
# iterative: bi CCS3project\AutogluonModels\ag-20241004_125408 ter CCS3project\AutogluonModels\ag-20241004_125424

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.15
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
CPU Count:          12
Memory Avail:       16.32 GB / 31.74 GB (51.4%)
Disk Space Avail:   142.35 GB / 476.93 GB (29.8%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "Autog

In [24]:
predictor_binary_default = TabularPredictor.load("AutogluonModels/ag-zscore-binary")
predictor_ternary_default = TabularPredictor.load("AutogluonModels/ag-zscore-ternary")
predictor_binary_pupil = TabularPredictor.load("AutogluonModels/ag-pupilzscore-binary")
predictor_ternary_pupil = TabularPredictor.load("AutogluonModels/ag-pupilzscore-ternary")

In [57]:
predictor_binary_default.leaderboard(test_data_binary, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,0.812709,0.806,accuracy,0.006033,0.001998,4.372923,0.006033,0.001998,4.372923,1,True,7
1,LightGBMLarge,0.802676,0.808,accuracy,0.024032,0.006507,2.605864,0.024032,0.006507,2.605864,1,True,13
2,LightGBM,0.785953,0.818,accuracy,0.006515,0.005514,0.872145,0.006515,0.005514,0.872145,1,True,4
3,WeightedEnsemble_L2,0.785953,0.818,accuracy,0.009024,0.005514,0.992101,0.002509,0.000000,0.119956,2,True,14
4,XGBoost,0.785953,0.814,accuracy,0.018774,0.005518,1.320716,0.018774,0.005518,1.320716,1,True,11
5,RandomForestGini,0.769231,0.744,accuracy,0.074576,0.047922,0.709944,0.074576,0.047922,0.709944,1,True,5
6,LightGBMXT,0.765886,0.780,accuracy,0.021046,0.006002,1.088507,0.021046,0.006002,1.088507,1,True,3
7,RandomForestEntr,0.759197,0.748,accuracy,0.073475,0.049515,0.725859,0.073475,0.049515,0.725859,1,True,6
8,ExtraTreesEntr,0.732441,0.700,accuracy,0.116702,0.047580,0.616526,0.116702,0.047580,0.616526,1,True,9
9,ExtraTreesGini,0.712375,0.700,accuracy,0.071514,0.063822,0.673442,0.071514,0.063822,0.673442,1,True,8


In [39]:
predictor_ternary_default.leaderboard(test_data_ternary, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM,0.715719,0.738,accuracy,0.062139,0.020538,3.697814,0.062139,0.020538,3.697814,1,True,5
1,WeightedEnsemble_L2,0.715719,0.738,accuracy,0.065655,0.021537,3.802269,0.003515,0.000999,0.104454,2,True,14
2,LightGBMLarge,0.705686,0.720,accuracy,0.076663,0.020290,5.465961,0.076663,0.020290,5.465961,1,True,13
3,LightGBMXT,0.685619,0.698,accuracy,0.020210,0.008515,1.613020,0.020210,0.008515,1.613020,1,True,4
4,XGBoost,0.665552,0.712,accuracy,0.032572,0.002000,2.114404,0.032572,0.002000,2.114404,1,True,11
5,CatBoost,0.638796,0.688,accuracy,0.006570,0.000999,4.261115,0.006570,0.000999,4.261115,1,True,8
6,RandomForestGini,0.632107,0.662,accuracy,0.091253,0.046440,0.723358,0.091253,0.046440,0.723358,1,True,6
7,RandomForestEntr,0.622074,0.658,accuracy,0.065909,0.048382,0.802906,0.065909,0.048382,0.802906,1,True,7
8,ExtraTreesEntr,0.575251,0.602,accuracy,0.086572,0.080158,0.634357,0.086572,0.080158,0.634357,1,True,10
9,ExtraTreesGini,0.558528,0.606,accuracy,0.083667,0.048093,0.741936,0.083667,0.048093,0.741936,1,True,9


In [58]:
feature_importance_binary = predictor_binary_default.feature_importance(test_data_binary)
print("Feature Importance:", feature_importance_binary)

Computing feature importance via permutation shuffling for 32 features using 299 rows with 5 shuffle sets...
	3.96s	= Expected runtime (0.79s per shuffle set)
	0.36s	= Actual runtime (Completed 5 of 5 shuffle sets)


Feature Importance:                                 importance    stddev   p_value  n  p99_high  \
Pupil Diameter Mean               0.076254  0.012151  0.000075  5  0.101273   
Saccade Duration Min              0.061538  0.008395  0.000041  5  0.078823   
Saccade Count                     0.035452  0.009036  0.000465  5  0.054057   
Fixation Duration Min             0.034783  0.020426  0.009489  5  0.076840   
Fixation Dispersion Min           0.010033  0.007844  0.022956  5  0.026183   
Saccade Duration Mean             0.009365  0.002798  0.000853  5  0.015126   
Fixation Duration Mean            0.008696  0.005596  0.012741  5  0.020219   
Saccade Peak Deceleration Min     0.006020  0.003664  0.010656  5  0.013564   
Fixation Count                    0.006020  0.007627  0.076159  5  0.021723   
Saccade Peak Deceleration Max     0.004682  0.003813  0.025803  5  0.012534   
Saccade Peak Acceleration Max     0.004013  0.005496  0.088904  5  0.015329   
Saccade Amplitude Min           

In [60]:
feature_importance_binary.to_csv('./results/feature_importance_binary.csv')

In [61]:
feature_importance_ternary = predictor_ternary_default.feature_importance(test_data_ternary)
print("Feature Importance:", feature_importance_ternary)

Computing feature importance via permutation shuffling for 32 features using 299 rows with 5 shuffle sets...
	13.76s	= Expected runtime (2.75s per shuffle set)
	1.38s	= Actual runtime (Completed 5 of 5 shuffle sets)


Feature Importance:                                 importance    stddev   p_value  n  p99_high  \
Pupil Diameter Mean               0.084281  0.014844  0.000111  5  0.114846   
Saccade Duration Min              0.071572  0.021232  0.000830  5  0.115288   
Fixation Duration Min             0.060201  0.033695  0.008098  5  0.129579   
Fixation Dispersion Mean          0.013378  0.005288  0.002406  5  0.024266   
Fixation Count                    0.011371  0.006520  0.008770  5  0.024795   
Fixation Duration Mean            0.010033  0.009751  0.041429  5  0.030110   
Saccade Duration Mean             0.008696  0.006076  0.016444  5  0.021205   
Saccade Duration Max              0.004682  0.002991  0.012448  5  0.010842   
Blink Duration Max                0.004013  0.004361  0.054350  5  0.012992   
Saccade Peak Acceleration Mean    0.003344  0.010308  0.254165  5  0.024570   
Saccade Amplitude Max             0.002007  0.010991  0.352000  5  0.024637   
Saccade Peak Velocity Mean      

In [62]:
feature_importance_ternary.to_csv('./results/feature_importance_ternary.csv')

In [63]:
a = feature_importance_binary[feature_importance_binary['p_value']<0.01].index.tolist()
b = feature_importance_ternary[feature_importance_ternary['p_value']<0.01].index.tolist()

In [64]:
a

['Pupil Diameter Mean',
 'Saccade Duration Min',
 'Saccade Count',
 'Fixation Duration Min',
 'Saccade Duration Mean']

In [65]:
b

['Pupil Diameter Mean',
 'Saccade Duration Min',
 'Fixation Duration Min',
 'Fixation Dispersion Mean',
 'Fixation Count']

In [30]:
c = set(b).union(set(a))
set(c)

{'Fixation Count',
 'Fixation Dispersion Mean',
 'Fixation Duration Min',
 'Pupil Diameter Mean',
 'Saccade Amplitude Min',
 'Saccade Count',
 'Saccade Duration Min'}

In [6]:
# Adding synthetic cognitive load labels to the feature dataframe for demonstration purposes
np.random.seed(42)  # For reproducibility

feature_df_binary_distill = feature_df_binary[['binary_label',
                                               'Fixation Count',
#  'Fixation Dispersion Mean',
 'Fixation Duration Min',
 'Pupil Diameter Mean',
#  'Saccade Count',
 'Saccade Duration Min'
 ]]

feature_df_ternary_distill = feature_df_ternary[['ternary_label',
                                                 'Fixation Count',
#  'Fixation Dispersion Mean',
 'Fixation Duration Min',
 'Pupil Diameter Mean',
#  'Saccade Count',
 'Saccade Duration Min'
 ]]

# Splitting the dataset into training and testing sets
train_data_binary, test_data_binary = train_test_split(feature_df_binary_distill, test_size=0.1, random_state=42)
train_data_ternary, test_data_ternary = train_test_split(feature_df_ternary_distill, test_size=0.1, random_state=42)

predictor_binary_default = TabularPredictor(label='binary_label', eval_metric='accuracy', verbosity=2,path = 'AutogluonModels/ag-pupilzscore-binary').fit(train_data_binary) #CCS3project\AutogluonModels\ag-20241004_080342
predictor_ternary_default = TabularPredictor(label='ternary_label', eval_metric='accuracy', verbosity=2,path = 'AutogluonModels/ag-pupilzscore-ternary').fit(train_data_ternary) #CCS3project\AutogluonModels\ag-20241004_080400
# predictor = TabularPredictor.load("AutogluonModels/ag-20241004_000434")
# normalized: bi CCS3project\AutogluonModels\ag-20241004_103135 ter  CCS3project\AutogluonModels\ag-20241004_103156
# normalized wo z score: bi CCS3project\AutogluonModels\ag-20241004_103835 ter CCS3project\AutogluonModels\ag-20241004_103854
# only z score: bi CCS3project\AutogluonModels\ag-20241004_104224 ter  CCS3project\AutogluonModels\ag-20241004_104245
# iterative: bi CCS3project\AutogluonModels\ag-20241004_125408 ter CCS3project\AutogluonModels\ag-20241004_125424

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.15
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
CPU Count:          12
Memory Avail:       16.11 GB / 31.74 GB (50.7%)
Disk Space Avail:   142.40 GB / 476.93 GB (29.9%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "Autog

[1000]	valid_set's binary_error: 0.208


	0.794	 = Validation score   (accuracy)
	1.18s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBM ...
	0.814	 = Validation score   (accuracy)
	0.48s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.784	 = Validation score   (accuracy)
	0.65s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.778	 = Validation score   (accuracy)
	0.78s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: CatBoost ...
	0.812	 = Validation score   (accuracy)
	2.03s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.758	 = Validation score   (accuracy)
	0.61s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.766	 = Validation score   (accuracy)
	0.68s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
d:\Applications\anaconda3\envs\ag_new\lib\site-packages\fastai\learner.py:53: FutureWa

In [83]:
best_model

'WeightedEnsemble_L2'

In [74]:
from io import StringIO

# Define the content as a string
content = """model	score_test	score_val	eval_metric	pred_time_test	pred_time_val	fit_time	pred_time_test_marginal	pred_time_val_marginal	fit_time_marginal	stack_level	can_infer	fit_order
0	XGBoost	0.668896	0.702	accuracy	0.044166	0.002000	0.726316	0.044166	0.002000	0.726316	1	True	11
1	RandomForestGini	0.668896	0.680	accuracy	0.076001	0.050597	0.646116	0.076001	0.050597	0.646116	1	True	6
2	WeightedEnsemble_L2	0.662207	0.710	accuracy	0.280986	0.182631	10.328972	0.002998	0.001017	0.106019	2	True	14
3	LightGBM	0.655518	0.690	accuracy	0.008000	0.005001	0.712404	0.008000	0.005001	0.712404	1	True	5
4	CatBoost	0.655518	0.702	accuracy	0.008000	0.001001	2.656607	0.008000	0.001001	2.656607	1	True	8
5	LightGBMXT	0.655518	0.682	accuracy	0.081992	0.048008	3.169395	0.081992	0.048008	3.169395	1	True	4
6	RandomForestEntr	0.655518	0.678	accuracy	0.087000	0.062999	0.820777	0.087000	0.062999	0.820777	1	True	7
7	LightGBMLarge	0.642140	0.696	accuracy	0.012002	0.004252	1.867059	0.012002	0.004252	1.867059	1	True	13
8	ExtraTreesEntr	0.628763	0.646	accuracy	0.112014	0.064958	0.657181	0.112014	0.064958	0.657181	1	True	10
9	ExtraTreesGini	0.608696	0.646	accuracy	0.098692	0.077346	0.718777	0.098692	0.077346	0.718777	1	True	9
10	NeuralNetTorch	0.588629	0.614	accuracy	0.009999	0.004919	6.887283	0.009999	0.004919	6.887283	1	True	12
11	NeuralNetFastAI	0.505017	0.578	accuracy	0.016995	0.014008	2.217653	0.016995	0.014008	2.217653	1	True	3
12	KNeighborsUnif	0.501672	0.522	accuracy	0.036007	0.018043	0.004001	0.036007	0.018043	0.004001	1	True	1
13	KNeighborsDist	0.498328	0.548	accuracy	0.032999	0.016026	0.003999	0.032999	0.016026	0.003999	1	True	2"""

# Use StringIO to simulate a file object
content_io = StringIO(content)

# Create the DataFrame
df = pd.read_csv(content_io, sep="\t")

# Display the DataFrame
df.to_csv('./results/feature_step3_ternary.csv')
df


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,XGBoost,0.668896,0.702,accuracy,0.044166,0.002000,0.726316,0.044166,0.002000,0.726316,1,True,11
1,RandomForestGini,0.668896,0.680,accuracy,0.076001,0.050597,0.646116,0.076001,0.050597,0.646116,1,True,6
2,WeightedEnsemble_L2,0.662207,0.710,accuracy,0.280986,0.182631,10.328972,0.002998,0.001017,0.106019,2,True,14
3,LightGBM,0.655518,0.690,accuracy,0.008000,0.005001,0.712404,0.008000,0.005001,0.712404,1,True,5
4,CatBoost,0.655518,0.702,accuracy,0.008000,0.001001,2.656607,0.008000,0.001001,2.656607,1,True,8
5,LightGBMXT,0.655518,0.682,accuracy,0.081992,0.048008,3.169395,0.081992,0.048008,3.169395,1,True,4
6,RandomForestEntr,0.655518,0.678,accuracy,0.087000,0.062999,0.820777,0.087000,0.062999,0.820777,1,True,7
7,LightGBMLarge,0.642140,0.696,accuracy,0.012002,0.004252,1.867059,0.012002,0.004252,1.867059,1,True,13
8,ExtraTreesEntr,0.628763,0.646,accuracy,0.112014,0.064958,0.657181,0.112014,0.064958,0.657181,1,True,10
9,ExtraTreesGini,0.608696,0.646,accuracy,0.098692,0.077346,0.718777,0.098692,0.077346,0.718777,1,True,9


In [13]:
predictor_binary_default.leaderboard(test_data_binary, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,0.812709,0.802,accuracy,0.018992,0.001000,1.312426,0.018992,0.001000,1.312426,1,True,7
1,LightGBMXT,0.806020,0.782,accuracy,0.015990,0.006998,0.974995,0.015990,0.006998,0.974995,1,True,3
2,RandomForestGini,0.799331,0.774,accuracy,0.069003,0.058001,0.622836,0.069003,0.058001,0.622836,1,True,5
3,LightGBMLarge,0.792642,0.788,accuracy,0.007001,0.004000,0.909863,0.007001,0.004000,0.909863,1,True,13
4,LightGBM,0.789298,0.810,accuracy,0.002000,0.004000,0.481982,0.002000,0.004000,0.481982,1,True,4
5,WeightedEnsemble_L2,0.789298,0.810,accuracy,0.003999,0.005033,0.590149,0.001999,0.001033,0.108167,2,True,14
6,XGBoost,0.782609,0.796,accuracy,0.019924,0.003001,0.497988,0.019924,0.003001,0.497988,1,True,11
7,RandomForestEntr,0.779264,0.772,accuracy,0.080908,0.047326,0.614558,0.080908,0.047326,0.614558,1,True,6
8,ExtraTreesGini,0.765886,0.752,accuracy,0.107585,0.060034,0.631605,0.107585,0.060034,0.631605,1,True,8
9,ExtraTreesEntr,0.749164,0.746,accuracy,0.074099,0.072570,0.624279,0.074099,0.072570,0.624279,1,True,9


In [21]:
predictor_binary_default.leaderboard(test_data_binary, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMLarge,0.809365,0.814,accuracy,0.007998,0.005001,0.817202,0.007998,0.005001,0.817202,1,True,13
1,CatBoost,0.806020,0.812,accuracy,0.005001,0.000996,2.176027,0.005001,0.000996,2.176027,1,True,7
2,LightGBM,0.799331,0.814,accuracy,0.004999,0.004003,0.526615,0.004999,0.004003,0.526615,1,True,4
3,XGBoost,0.795987,0.800,accuracy,0.025470,0.002001,0.418689,0.025470,0.002001,0.418689,1,True,11
4,RandomForestGini,0.789298,0.784,accuracy,0.072999,0.051897,0.622599,0.072999,0.051897,0.622599,1,True,5
5,WeightedEnsemble_L2,0.789298,0.822,accuracy,0.246883,0.090385,12.480349,0.003997,0.000000,0.114642,2,True,14
6,RandomForestEntr,0.785953,0.778,accuracy,0.060005,0.049007,0.637169,0.060005,0.049007,0.637169,1,True,6
7,LightGBMXT,0.772575,0.794,accuracy,0.017999,0.010000,1.174378,0.017999,0.010000,1.174378,1,True,3
8,NeuralNetTorch,0.765886,0.750,accuracy,0.019936,0.005028,8.985644,0.019936,0.005028,8.985644,1,True,12
9,ExtraTreesGini,0.752508,0.758,accuracy,0.070264,0.056444,0.905633,0.070264,0.056444,0.905633,1,True,8


In [24]:
predictor_binary_default.leaderboard(test_data_binary, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM,0.802676,0.794,accuracy,0.003002,0.004004,0.453555,0.003002,0.004004,0.453555,1,True,4
1,WeightedEnsemble_L2,0.802676,0.794,accuracy,0.004999,0.004004,0.575430,0.001997,0.000000,0.121875,2,True,14
2,XGBoost,0.799331,0.778,accuracy,0.011000,0.002000,0.365412,0.011000,0.002000,0.365412,1,True,11
3,CatBoost,0.795987,0.792,accuracy,0.004995,0.000997,1.501873,0.004995,0.000997,1.501873,1,True,7
4,LightGBMLarge,0.775920,0.770,accuracy,0.009001,0.003965,0.939851,0.009001,0.003965,0.939851,1,True,13
5,LightGBMXT,0.765886,0.748,accuracy,0.014998,0.008000,1.039885,0.014998,0.008000,1.039885,1,True,3
6,RandomForestGini,0.762542,0.746,accuracy,0.082999,0.050177,0.610380,0.082999,0.050177,0.610380,1,True,5
7,RandomForestEntr,0.762542,0.746,accuracy,0.083019,0.046991,0.584679,0.083019,0.046991,0.584679,1,True,6
8,NeuralNetTorch,0.715719,0.694,accuracy,0.009000,0.005995,5.471296,0.009000,0.005995,5.471296,1,True,12
9,ExtraTreesGini,0.715719,0.726,accuracy,0.088987,0.047938,0.549792,0.088987,0.047938,0.549792,1,True,8


In [5]:
predictor_binary_default.leaderboard(test_data_binary, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMLarge,0.806020,0.794,accuracy,0.006998,0.004998,0.871820,0.006998,0.004998,0.871820,1,True,13
1,LightGBM,0.802676,0.798,accuracy,0.004248,0.004214,0.441124,0.004248,0.004214,0.441124,1,True,4
2,WeightedEnsemble_L2,0.802676,0.798,accuracy,0.006248,0.004214,0.562648,0.002000,0.000000,0.121524,2,True,14
3,LightGBMXT,0.799331,0.786,accuracy,0.021995,0.009000,4.010950,0.021995,0.009000,4.010950,1,True,3
4,XGBoost,0.795987,0.798,accuracy,0.022476,0.003000,0.990109,0.022476,0.003000,0.990109,1,True,11
5,RandomForestGini,0.795987,0.776,accuracy,0.082808,0.048889,1.068890,0.082808,0.048889,1.068890,1,True,5
6,CatBoost,0.792642,0.798,accuracy,0.008120,0.001999,6.932680,0.008120,0.001999,6.932680,1,True,7
7,RandomForestEntr,0.789298,0.770,accuracy,0.073368,0.049258,0.725944,0.073368,0.049258,0.725944,1,True,6
8,ExtraTreesEntr,0.789298,0.746,accuracy,0.108311,0.047496,0.736780,0.108311,0.047496,0.736780,1,True,9
9,ExtraTreesGini,0.765886,0.744,accuracy,0.098183,0.045060,0.655372,0.098183,0.045060,0.655372,1,True,8


In [7]:
predictor_binary_default.leaderboard(test_data_binary, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMLarge,0.809365,0.814,accuracy,0.007522,0.005001,0.800846,0.007522,0.005001,0.800846,1,True,13
1,CatBoost,0.806020,0.812,accuracy,0.009571,0.001009,2.032361,0.009571,0.001009,2.032361,1,True,7
2,LightGBM,0.799331,0.814,accuracy,0.003998,0.004001,0.478612,0.003998,0.004001,0.478612,1,True,4
3,XGBoost,0.795987,0.800,accuracy,0.028807,0.002000,0.450592,0.028807,0.002000,0.450592,1,True,11
4,RandomForestGini,0.789298,0.784,accuracy,0.080137,0.096521,0.648649,0.080137,0.096521,0.648649,1,True,5
5,WeightedEnsemble_L2,0.789298,0.822,accuracy,0.195388,0.100888,13.323472,0.003996,0.000000,0.126539,2,True,14
6,RandomForestEntr,0.785953,0.778,accuracy,0.077029,0.050482,0.782978,0.077029,0.050482,0.782978,1,True,6
7,LightGBMXT,0.772575,0.794,accuracy,0.013996,0.009000,1.182658,0.013996,0.009000,1.182658,1,True,3
8,NeuralNetTorch,0.765886,0.750,accuracy,0.027003,0.005834,9.996567,0.027003,0.005834,9.996567,1,True,12
9,ExtraTreesEntr,0.752508,0.766,accuracy,0.095805,0.073043,0.684397,0.095805,0.073043,0.684397,1,True,9


In [ ]:
predictor_binary_default.leaderboard(test_data_binary, silent=True)

In [14]:
predictor_ternary_default.leaderboard(test_data_ternary, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,XGBoost,0.705686,0.732,accuracy,0.044006,0.002997,0.808740,0.044006,0.002997,0.808740,1,True,11
1,WeightedEnsemble_L2,0.695652,0.736,accuracy,0.165493,0.082458,4.133821,0.002999,0.000000,0.121443,2,True,14
2,LightGBM,0.688963,0.720,accuracy,0.011996,0.007999,0.898501,0.011996,0.007999,0.898501,1,True,5
3,CatBoost,0.675585,0.718,accuracy,0.007974,0.002002,2.572761,0.007974,0.002002,2.572761,1,True,8
4,LightGBMLarge,0.665552,0.712,accuracy,0.008000,0.004001,1.692622,0.008000,0.004001,1.692622,1,True,13
5,RandomForestGini,0.648829,0.712,accuracy,0.074005,0.049117,0.669445,0.074005,0.049117,0.669445,1,True,6
6,LightGBMXT,0.648829,0.710,accuracy,0.074908,0.020001,2.039125,0.074908,0.020001,2.039125,1,True,4
7,RandomForestEntr,0.642140,0.712,accuracy,0.086498,0.049002,0.632688,0.086498,0.049002,0.632688,1,True,7
8,ExtraTreesGini,0.628763,0.660,accuracy,0.088475,0.049998,0.635645,0.088475,0.049998,0.635645,1,True,9
9,ExtraTreesEntr,0.615385,0.658,accuracy,0.091035,0.058562,0.624878,0.091035,0.058562,0.624878,1,True,10


In [22]:
predictor_ternary_default.leaderboard(test_data_ternary, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.705686,0.738,accuracy,0.172284,0.098612,4.740486,0.002999,0.000000,0.128502,2,True,14
1,XGBoost,0.702341,0.724,accuracy,0.040999,0.002003,0.737566,0.040999,0.002003,0.737566,1,True,11
2,CatBoost,0.695652,0.730,accuracy,0.008985,0.002004,3.411042,0.008985,0.002004,3.411042,1,True,8
3,RandomForestEntr,0.695652,0.700,accuracy,0.084018,0.050001,0.705873,0.084018,0.050001,0.705873,1,True,7
4,LightGBMXT,0.692308,0.700,accuracy,0.036994,0.008000,0.981889,0.036994,0.008000,0.981889,1,True,4
5,RandomForestGini,0.688963,0.716,accuracy,0.069997,0.046494,0.618515,0.069997,0.046494,0.618515,1,True,6
6,ExtraTreesGini,0.682274,0.680,accuracy,0.090303,0.050113,0.582427,0.090303,0.050113,0.582427,1,True,9
7,LightGBM,0.678930,0.726,accuracy,0.020001,0.008000,0.920791,0.020001,0.008000,0.920791,1,True,5
8,ExtraTreesEntr,0.678930,0.674,accuracy,0.093998,0.048180,0.570518,0.093998,0.048180,0.570518,1,True,10
9,LightGBMLarge,0.672241,0.708,accuracy,0.008999,0.004001,1.636882,0.008999,0.004001,1.636882,1,True,13


In [25]:
predictor_ternary_default.leaderboard(test_data_ternary, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,XGBoost,0.668896,0.702,accuracy,0.044166,0.002000,0.726316,0.044166,0.002000,0.726316,1,True,11
1,RandomForestGini,0.668896,0.680,accuracy,0.076001,0.050597,0.646116,0.076001,0.050597,0.646116,1,True,6
2,WeightedEnsemble_L2,0.662207,0.710,accuracy,0.280986,0.182631,10.328972,0.002998,0.001017,0.106019,2,True,14
3,LightGBM,0.655518,0.690,accuracy,0.008000,0.005001,0.712404,0.008000,0.005001,0.712404,1,True,5
4,CatBoost,0.655518,0.702,accuracy,0.008000,0.001001,2.656607,0.008000,0.001001,2.656607,1,True,8
5,LightGBMXT,0.655518,0.682,accuracy,0.081992,0.048008,3.169395,0.081992,0.048008,3.169395,1,True,4
6,RandomForestEntr,0.655518,0.678,accuracy,0.087000,0.062999,0.820777,0.087000,0.062999,0.820777,1,True,7
7,LightGBMLarge,0.642140,0.696,accuracy,0.012002,0.004252,1.867059,0.012002,0.004252,1.867059,1,True,13
8,ExtraTreesEntr,0.628763,0.646,accuracy,0.112014,0.064958,0.657181,0.112014,0.064958,0.657181,1,True,10
9,ExtraTreesGini,0.608696,0.646,accuracy,0.098692,0.077346,0.718777,0.098692,0.077346,0.718777,1,True,9


In [15]:
feature_importance_binary = predictor_binary_default.feature_importance(test_data_binary)
print("Feature Importance:", feature_importance_binary)

Computing feature importance via permutation shuffling for 5 features using 299 rows with 5 shuffle sets...
	0.43s	= Expected runtime (0.09s per shuffle set)
	0.11s	= Actual runtime (Completed 5 of 5 shuffle sets)


Feature Importance:                        importance    stddev   p_value  n  p99_high   p99_low
Pupil Diameter Mean      0.101003  0.011919  0.000023  5  0.125544  0.076462
Saccade Duration Min     0.058863  0.006935  0.000023  5  0.073143  0.044583
Saccade Count            0.058194  0.024983  0.003239  5  0.109634  0.006753
Fixation Duration Min    0.052174  0.023364  0.003763  5  0.100280  0.004068
Fixation Count           0.018729  0.007327  0.002318  5  0.033816  0.003642


In [16]:
feature_importance_ternary = predictor_ternary_default.feature_importance(test_data_ternary)
print("Feature Importance:", feature_importance_ternary)

Computing feature importance via permutation shuffling for 5 features using 299 rows with 5 shuffle sets...
	5.72s	= Expected runtime (1.14s per shuffle set)
	1.15s	= Actual runtime (Completed 5 of 5 shuffle sets)


Feature Importance:                        importance    stddev   p_value  n  p99_high   p99_low
Pupil Diameter Mean      0.143813  0.013790  0.000010  5  0.172206  0.115420
Saccade Duration Min     0.113043  0.020940  0.000135  5  0.156159  0.069928
Fixation Duration Min    0.076923  0.015143  0.000171  5  0.108102  0.045744
Saccade Count            0.055518  0.014693  0.000537  5  0.085771  0.025266
Fixation Count           0.048829  0.007700  0.000072  5  0.064683  0.032976


In [11]:
#from 180701
data = feature_df_binary_distill
label = 'binary_label'
kf = KFold(n_splits=10, shuffle=True, random_state=42)
fold = 1

# Iterate through each fold
for train_index, val_index in kf.split(data):
    train_data = data.iloc[train_index]
    val_data = data.iloc[val_index]
    
    # Train the model on the training fold
    predictor_binary = TabularPredictor(label=label, eval_metric='accuracy', verbosity=2).fit(train_data)
    
    predictor_binary.leaderboard(val_data, silent=True).to_csv('./results/10foldbinary_reduced/leaderboard_binary_fold'+str(fold)+'.csv')
    

    
    print(f"Fold {fold}")
    fold += 1


No path specified. Models will be saved in: "AutogluonModels\ag-20241229_145905"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.15
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
CPU Count:          12
Memory Avail:       16.49 GB / 31.74 GB (52.0%)
Disk Space Avail:   140.94 GB / 476.93 GB (29.6%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial p

[1000]	valid_set's binary_error: 0.206


	0.802	 = Validation score   (accuracy)
	1.48s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBM ...
	0.812	 = Validation score   (accuracy)
	0.73s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.794	 = Validation score   (accuracy)
	0.64s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.794	 = Validation score   (accuracy)
	0.61s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: CatBoost ...
	0.82	 = Validation score   (accuracy)
	2.18s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.774	 = Validation score   (accuracy)
	0.69s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.764	 = Validation score   (accuracy)
	0.62s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
d:\Applications\anaconda3\envs\ag_new\lib\site-packages\fastai\learner.py:53: FutureW

Fold 1
[1000]	valid_set's binary_error: 0.2


	0.804	 = Validation score   (accuracy)
	1.02s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBM ...
	0.81	 = Validation score   (accuracy)
	0.5s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.778	 = Validation score   (accuracy)
	0.6s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.79	 = Validation score   (accuracy)
	0.63s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: CatBoost ...
	0.8	 = Validation score   (accuracy)
	1.18s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.734	 = Validation score   (accuracy)
	0.79s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.738	 = Validation score   (accuracy)
	0.74s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
d:\Applications\anaconda3\envs\ag_new\lib\site-packages\fastai\learner.py:53: FutureWarning

...

Fold 9


	0.802	 = Validation score   (accuracy)
	0.86s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's binary_error: 0.206


	0.812	 = Validation score   (accuracy)
	0.45s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.786	 = Validation score   (accuracy)
	0.62s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.788	 = Validation score   (accuracy)
	0.59s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: CatBoost ...
	0.818	 = Validation score   (accuracy)
	1.72s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.754	 = Validation score   (accuracy)
	0.75s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.76	 = Validation score   (accuracy)
	0.8s	 = Training   runtime
	0.13s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
d:\Applications\anaconda3\envs\ag_new\lib\site-packages\fastai\learner.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module i

Fold 10


In [12]:

data = feature_df_ternary_distill
label = 'ternary_label'
kf = KFold(n_splits=10, shuffle=True, random_state=42)
fold = 1

# Iterate through each fold
for train_index, val_index in kf.split(data):
    train_data = data.iloc[train_index]
    val_data = data.iloc[val_index]
    
    # Train the model on the training fold
    predictor_ternary = TabularPredictor(label=label, eval_metric='accuracy', verbosity=2).fit(train_data)
    
    predictor_ternary.leaderboard(val_data, silent=True).to_csv('./results/10foldternary_reduced/leaderboard_fold'+str(fold)+'.csv')
    

    
    print(f"Fold {fold}")
    fold += 1


No path specified. Models will be saved in: "AutogluonModels\ag-20241229_150217"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.15
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
CPU Count:          12
Memory Avail:       16.36 GB / 31.74 GB (51.6%)
Disk Space Avail:   139.84 GB / 476.93 GB (29.3%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial p

[1000]	valid_set's multi_error: 0.332


	0.682	 = Validation score   (accuracy)
	1.62s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: LightGBM ...
	0.67	 = Validation score   (accuracy)
	0.95s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.668	 = Validation score   (accuracy)
	0.66s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.67	 = Validation score   (accuracy)
	0.68s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: CatBoost ...
	0.676	 = Validation score   (accuracy)
	2.58s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.638	 = Validation score   (accuracy)
	0.7s	 = Training   runtime
	0.12s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.64	 = Validation score   (accuracy)
	0.72s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: XGBoost ...
	0.68	 = Validation score   (accuracy)
	0.81s	 = Training   runtime
	0.0s	 = Validation runtime


Fold 1


d:\Applications\anaconda3\envs\ag_new\lib\site-packages\fastai\learner.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(file, map_location=device, **t

...

Fold 8


d:\Applications\anaconda3\envs\ag_new\lib\site-packages\fastai\learner.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(file, map_location=device, **t

Fold 9


d:\Applications\anaconda3\envs\ag_new\lib\site-packages\fastai\learner.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(file, map_location=device, **t

Fold 10


In [15]:
#from 134746-135554 
data = feature_df.drop(columns=['Label','Level','ternary_label'])
data = data[['binary_label','Participant',
                                               'Fixation Count',
#  'Fixation Dispersion Mean',
 'Fixation Duration Min',
 'Pupil Diameter Mean',
#  'Saccade Count',
 'Saccade Duration Min'
 ]]
label = 'binary_label'
participants = data['Participant'].unique()
sub = 1

# LOSO cross-validation
for participant in participants:
    train_data = data[data['Participant'] != participant].drop(columns=['Participant'])
    val_data = data[data['Participant'] == participant].drop(columns=['Participant'])
    
    # Train the model on the training data excluding one participant
    predictor = TabularPredictor(label=label, eval_metric='accuracy', verbosity=2).fit(train_data)
    
    predictor.leaderboard(val_data, silent=True).to_csv('./results/losobinary_reduced/leaderboard_sub'+str(sub)+'.csv')
    
    print(f"Fold {sub}")
    sub += 1
    
    

No path specified. Models will be saved in: "AutogluonModels\ag-20241229_151326"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.15
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
CPU Count:          12
Memory Avail:       17.33 GB / 31.74 GB (54.6%)
Disk Space Avail:   138.18 GB / 476.93 GB (29.0%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial p

[1000]	valid_set's binary_error: 0.24


	0.808	 = Validation score   (accuracy)
	0.6s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.776	 = Validation score   (accuracy)
	0.74s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.776	 = Validation score   (accuracy)
	0.64s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: CatBoost ...
	0.81	 = Validation score   (accuracy)
	2.75s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.726	 = Validation score   (accuracy)
	0.71s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.74	 = Validation score   (accuracy)
	0.69s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
No improvement since epoch 6: early stopping
d:\Applications\anaconda3\envs\ag_new\lib\site-packages\fastai\learner.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default va

Fold 1


	0.788	 = Validation score   (accuracy)
	0.87s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBM ...
	0.796	 = Validation score   (accuracy)
	0.5s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.77	 = Validation score   (accuracy)
	0.65s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.762	 = Validation score   (accuracy)
	0.62s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: CatBoost ...
	0.798	 = Validation score   (accuracy)
	1.3s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.756	 = Validation score   (accuracy)
	0.63s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.75	 = Validation score   (accuracy)
	0.58s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
d:\Applications\anaconda3\envs\ag_new\lib\site-packages\fastai\learner.py:53: FutureWarni

...

Fold 19


	0.01s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: KNeighborsDist ...
	0.664	 = Validation score   (accuracy)
	0.01s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: LightGBMXT ...
	0.802	 = Validation score   (accuracy)
	1.4s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBM ...
	0.794	 = Validation score   (accuracy)
	1.08s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.772	 = Validation score   (accuracy)
	0.98s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.788	 = Validation score   (accuracy)
	1.18s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: CatBoost ...
	0.812	 = Validation score   (accuracy)
	2.22s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.758	 = Validation score   (accuracy)
	0.96s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: ExtraTreesEntr ...


Fold 20


	0.658	 = Validation score   (accuracy)
	0.01s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: LightGBMXT ...
	0.792	 = Validation score   (accuracy)
	1.75s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBM ...
	0.79	 = Validation score   (accuracy)
	1.06s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.766	 = Validation score   (accuracy)
	0.87s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.754	 = Validation score   (accuracy)
	1.05s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: CatBoost ...
	0.782	 = Validation score   (accuracy)
	1.49s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.766	 = Validation score   (accuracy)
	0.9s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.766	 = Validation score   (accuracy)
	1.06s	 = Training   runtime
	0.08s	 = Validation r

Fold 21


In [16]:
#from 134746-135554 
data = feature_df.drop(columns=['Label','Level','binary_label'])
data = data[['ternary_label','Participant',
                                               'Fixation Count',
#  'Fixation Dispersion Mean',
 'Fixation Duration Min',
 'Pupil Diameter Mean',
#  'Saccade Count',
 'Saccade Duration Min'
 ]]
label = 'ternary_label'
participants = data['Participant'].unique()
sub = 1

# LOSO cross-validation
for participant in participants:
    train_data = data[data['Participant'] != participant].drop(columns=['Participant'])
    val_data = data[data['Participant'] == participant].drop(columns=['Participant'])
    
    # Train the model on the training data excluding one participant
    predictor = TabularPredictor(label=label, eval_metric='accuracy', verbosity=2).fit(train_data)
    
    predictor.leaderboard(val_data, silent=True).to_csv('./results/losoternary_reduced/leaderboard_sub'+str(sub)+'.csv')
    
    print(f"Fold {sub}")
    sub += 1
    
    

No path specified. Models will be saved in: "AutogluonModels\ag-20241229_152151"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.15
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
CPU Count:          12
Memory Avail:       16.17 GB / 31.74 GB (50.9%)
Disk Space Avail:   135.78 GB / 476.93 GB (28.5%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial p

[1000]	valid_set's multi_error: 0.306


	0.702	 = Validation score   (accuracy)
	3.78s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: LightGBM ...
	0.734	 = Validation score   (accuracy)
	2.48s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.716	 = Validation score   (accuracy)
	0.92s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.718	 = Validation score   (accuracy)
	1.29s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: CatBoost ...
	0.722	 = Validation score   (accuracy)
	6.16s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.676	 = Validation score   (accuracy)
	1.35s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.676	 = Validation score   (accuracy)
	1.38s	 = Training   runtime
	0.13s	 = Validation runtime
Fitting model: XGBoost ...
	0.714	 = Validation score   (accuracy)
	1.74s	 = Training   runtime
	0.01s	 = Validation run

Fold 1


User-specified model hyperparameters to be fit:
{
	'NN_TORCH': {},
	'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}, {}, 'GBMLarge'],
	'CAT': {},
	'XGB': {},
	'FASTAI': {},
	'RF': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
	'XT': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
	'KNN': [{'weights': 'uniform', 'ag_args': {'name_suffix': 'Unif'}}, {'weights': 'distance', 'ag_args': {'name_suffix': 'Dist'}}],
}
Fitting 13 L1 models .

...

Fold 19


d:\Applications\anaconda3\envs\ag_new\lib\site-packages\fastai\learner.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(file, map_location=device, **t

Fold 20


d:\Applications\anaconda3\envs\ag_new\lib\site-packages\fastai\learner.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(file, map_location=device, **t

Fold 21
